<a href="https://colab.research.google.com/github/edtrawtmam/hsp/blob/main/Calculadora%20HSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Desenha cada molécula com os dados atribuidos
# Cria um dataframe das substâncias
# Plota gráficos minimos.
# Modificado por Ed Trawtmam em 15/06/2021
############################################################################################################################

# Calcula o HSP segundo Mathieu 2018.
#!/usr/bin/python
# -*- coding: utf-8 -*-

# script file to compute HSP components of pure compounds
# note that the rdkit library is required
# input: MDL .mol files
# output: delta(d) delta(p) delta(h) Vm are printed out in this order
# units: MPa**1/2 and cc/mol

In [3]:
# Install RDKit. Takes 2-3 minutes
# From https://neelcrew.blogspot.com/2021/01/installing-rdkit-in-google-colaboratory.html . Acesado em 11/06/2021.

import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO


logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)


def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """

    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )

    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)

    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return

        logger.info("force re-install")

    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)

    logger.info("python version: {}".format(python_version))

    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)

    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')

    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')

    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python=={}".format(python_version),
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")

    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))


if __name__ == "__main__":
    install()

add /root/miniconda/lib/python3.7/site-packages to PYTHONPATH
python version: 3.7.10
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit
done
rdkit-2020.09.1 installation finished!


In [2]:
# Para plotar gráficos... será?
!pip install plotly==4.12.0

     |████████████████████████████████| 13.1MB 216kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [4]:
import sys, os, fileinput
from math import sqrt
from collections import Counter, OrderedDict, defaultdict
from rdkit import Chem
from rdkit.Chem import rdDepictor
rdDepictor.SetPreferCoordGen(True)
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
pd.options.plotting.backend = 'plotly'
from rdkit.Chem import PandasTools
import plotly.express as px
import plotly.graph_objects as go # Talvez inutil
from plotly.subplots import make_subplots


In [5]:
# FOR MOLAR VOLUME AND REFRACTIVITY
# additive increments
params = (('B30', 7.61, 2.91),
          ('Br10', 27.52, 8.44),
          ('C10', 14.68, 2.32),
          ('C20', 9.89, 4.01),
          ('C21', 22.77, 4.58),
          ('C30', -0.00, 3.15),
          ('C30a', -0.00, 3.48),
          ('C31', 13.23, 4.65),
          ('C31a', 13.23, 4.46),
          ('C32', 27.17, 5.38),
          ('C40', -8.40, 2.60),
          ('C41', 4.46, 3.48),
          ('C42', 16.57, 4.60),
          ('C43', 29.58, 5.74),
          ('Cl10', 24.74, 5.87),
          ('F10', 17.63, 1.06),
          ('Ge40', 9.36, 8.54),
          ('I10', 35.64, 13.75),
          ('N10', 14.09, 1.55),
          ('N20', 7.42, 2.60),
          ('N20a', 7.42, 2.44),
          ('N21', 18.14, 3.55),
          ('N30', -3.08, 2.89),
          ('N30a', -3.08, 2.85),
          ('N31', 7.74, 3.69),
          ('N31a', 7.74, 3.72),
          ('N32', 17.81, 4.60),
          ('N43', 5.36, 8.02),
          ('O10', 14.89, 1.84),
          ('O20', 6.25, 1.55),
          ('O20a', 6.25, 0.71),
          ('O21', 11.78, 2.51),
          ('P30', 10.42, 7.41),
          ('P40', -1.94, 4.98),
          ('P41', 10.06, 5.41),
          ('R5', 9.41, None),
          ('R6', 6.89, None),
          ('R<5', 10.89, None),
          ('R>6', 3.75, None),
          ('S10', 25.92, 10.60),
          ('S20', 14.90, 8.22),
          ('S20a', 14.90, 7.05),
          ('S21', 26.14, 8.71),
          ('S30', 5.58, 7.28),
          ('S40', -3.74, 5.12),
          ('Se20', 19.00, 11.53),
          ('Se20a', 19.00, 9.43),
          ('Si40', 9.28, 6.38),
          ('Si41', 23.35, 7.85),
          ('Sn40', 14.47, 14.90),
          ('Ti40', 6.09, 14.66),
          ('aromat', 1.82, None))
vi = dict((p[0], p[1]) for p in params)  # volume increments
ri = dict((p[0], p[2]) for p in params)  # molar refractivity increments

# FOR POLAR (P) HSP COMPONENT
params_p = {
    'N(1)': 2783,
    'N(2)': 8235,
    'O(0)': 1603,
    'O(1)': 4125,
    'Cl(0)': 1637,
    'C=O': 7492,
    'COOH': -5494,
    'COinAmide': 15972,
    'Carbonate': 19019,
    'Ester': 3653,
    'C#N': 16056,
    'NitroOnC': 13276,
    'O=P': 20310,
}

POLARGROUPS = OrderedDict([
  ("COOH", ("[CX3](=O)[OX2H1]"                          , (1,2))),  # =O in carboxilic acid
  ("NitroOnC", ("[#6][$([NX3](=O)=O),$([NX3+](=O)[O-])]", (1,))),  # N in nitroaliphatic
  ("Carbonate", ("[OX2;R][CX3;R](=O)[OX2]"              , (0,1,3))),  # Carbonate
  ("Ester", ("[OX2][CX3]=O"                             , (1,))),  # C in cyclic ester
  ("COinAmide", ("[NX3][CX3](=[OX1])[#6]"               , (1,))),
  ("SO2", ("O=S=O"                                      , (1,))),  # S in sulfone
])

# FOR HYDROGEN-BONDING (H) HSP COMPONENT
params_h = {
    'HC': 24.5,
    'HN': -1576,
    'HNamide': 5060,
    'H2N': 5484,
    'HO': 16945,
    'HO_COOH': 7094,
    'N': 3252,
    'O': 1980,
    'X': 412,
}

In [6]:
def get_ring_descriptors(mol, maxi=6, mini=5):
    """ return dict of ring descriptors for molecule provided as input """
    dic = Counter()
    ri = mol.GetRingInfo()
    for ring in ri.AtomRings():
        size = len(ring)
        if size > maxi:
            label = 'R>'+str(maxi)
        elif size < mini:
            label = 'R<'+str(mini)
        else:
            label = 'R%u' %len(ring)
        dic[label] += 1
        # contribute also +1 aromatic ring ?
        atoms = [mol.GetAtomWithIdx(i) for i in ring]
        if all(at.GetIsAromatic() for at in atoms):
            dic['aromat'] += 1
    return dic

def GetBChar(bond):
    if bond.GetBondType()==Chem.rdchem.BondType.AROMATIC: return '~'
    if bond.GetBondType()==Chem.rdchem.BondType.DOUBLE: return '='
    if bond.GetBondType()==Chem.rdchem.BondType.TRIPLE: return '#'
    return '-'

def get_nH(atom):
    nD = len([x for x in atom.GetNeighbors() if x.GetMass()>2 and x.GetSymbol()=='H'])
    nH = nD + atom.GetTotalNumHs()
    return nH

def isNitroN(at, mol):
    if at.GetSymbol() != "N": return False
    if at.GetTotalDegree() != 3: return False
    voisins = at.GetNeighbors()
    Os = [a for a in voisins if a.GetSymbol()=="O"]
    O1s = [a for a in Os if a.GetTotalDegree()==1]
    return len(O1s) > 1
    
def isAmideN(at, mol):
    amideSMARTS = "[NX3][CX3](=[OX1])[#6]"
    amidePattern = Chem.MolFromSmarts(amideSMARTS)
    N_indices = [t[0] for t in mol.GetSubstructMatches(amidePattern)]
    return at.GetIdx() in N_indices
    
def inCOOH(at, mol):
    acSMARTS = "[CX3](=O)[OX2H1]"
    acPattern = Chem.MolFromSmarts(acSMARTS)
    OH_indices = [t[2] for t in mol.GetSubstructMatches(acPattern)]
    return at.GetIdx() in OH_indices

In [7]:
def get_polar_groups(mol):
    global POLARGROUPS
    by_type = defaultdict(list)
    counted_indices = set()
    # first count complex polar groups
    for group_name in POLARGROUPS:
        group, positions = POLARGROUPS[group_name]
        pattern = Chem.MolFromSmarts(group)
        tuples = mol.GetSubstructMatches(pattern)
        for tup in tuples:
            if set(tup) & counted_indices: continue
            counted_indices |= set(tup)
            by_type[group_name].append(1)
    # count insaturated polar bonds
    for bond in mol.GetBonds():
        order = GetBChar(bond)
        if order in ("#", "=", "~"):
            abeg, aend = bond.GetBeginAtom(), bond.GetEndAtom()
            tup = (abeg.GetIdx(), aend.GetIdx())
            if set(tup) & counted_indices: continue
            symbols = sorted([abeg.GetSymbol(), aend.GetSymbol()])
            if set(symbols) == set(["C"]): continue
            bondsymbol = order.join(symbols)
            counted_indices |= set(tup)
            by_type[bondsymbol].append(order)
    # count saturated heteroatoms
    for hetat in mol.GetAtoms():
        idx = hetat.GetIdx()
        if idx in counted_indices: continue
        coo =  hetat.GetTotalDegree()
        symbol = hetat.GetSymbol()
        if symbol == "C": continue
        if symbol == "P" and coo > 3: continue
        name = "%s(%u)" %(symbol, get_nH(hetat))
        if name in ("N(0)", "F(0)"): continue
        counted_indices.add(idx)
        by_type[name].append(idx)
    return dict((group_name, len(by_type[group_name])) for group_name in by_type)


def get_polar_groups(mol):
    global POLARGROUPS
    by_type = defaultdict(list)
    counted_indices = set()
    for group_name in POLARGROUPS:
        group, positions = POLARGROUPS[group_name]
        pattern = Chem.MolFromSmarts(group)
        tuples = mol.GetSubstructMatches(pattern)
        for tup in tuples:
            pos = positions[0]
            atomindex = tup[pos]
            ##>> if set(tup) & counted_indices: continue
            counted_indices |= set(tup)
            by_type[group_name].append(atomindex)
    for bond in mol.GetBonds():
        order = GetBChar(bond)
        if order in ("#", "=", "~"):
            abeg, aend = bond.GetBeginAtom(), bond.GetEndAtom()
            symbols = sorted([abeg.GetSymbol(), aend.GetSymbol()])
            if symbols[0] == symbols[1]: continue  # ADDED to remove C=C and C~C
            tup = (abeg.GetIdx(), aend.GetIdx())
            if set(tup) & counted_indices: continue
            bondsymbol = order.join(symbols)
            counted_indices |= set(tup)
            by_type[bondsymbol].append(order)
    for hetat in mol.GetAtoms():
        idx = hetat.GetIdx()
        ##>> if idx in counted_indices: continue
        coo =  hetat.GetTotalDegree()
        symbol = hetat.GetSymbol()
        #if symbol == "C":
            #voisins = hetat.GetNeighbors()
            #Fs = [v for v in voisins if v.GetSymbol()=="F"]
            #if len(Fs) == 3:
                #by_type["CF3"].append(idx)
        if symbol == "C": continue
        if symbol == "P" and coo > 3: continue
        name = "%s(%u)" %(symbol, get_nH(hetat))
        if name in ("N(0)", "F(0)"): continue
        counted_indices.add(idx)
        by_type[name].append(idx)
 #   print("Grupos para dp:", Counter(dict((group_name, len(by_type[group_name])) for group_name in by_type)))
    return dict((group_name, len(by_type[group_name])) for group_name in by_type)



In [8]:
def get_dic_h(mol):
    dic = defaultdict(int)
    for at in mol.GetAtoms():
        symbol = at.GetSymbol()
        coo = at.GetTotalDegree()
        voisins = [v for v in at.GetNeighbors()]
        if symbol in ("N", "O"):
            if not isNitroN(at, mol):
                dic[symbol] += 1
        if symbol in ("F", "Cl", "Br", "I"):
            dic["X"] += 1
        nH = get_nH(at)
        if nH == 0: continue
        if symbol == "C":
            dic["HC"] += nH
            continue
        if symbol == "N":
            if isAmideN(at, mol):
                dic["HNamide"] += nH
                continue
            if nH == 2:
                dic['H2N'] += 2
            elif nH == 1:
                dic["HN"] += 1
            continue
        if symbol == "O":
            if inCOOH(at, mol):
                dic["HO_COOH"] += nH
            else:
                dic["HO"] += nH
  #  print("Grupos para dh:", dict(Counter(dic)))
    return dic
  
def get_HSPp(mol):
    d = get_polar_groups(mol)
    if set(d) - set(params_p): return float(0)
    eP = sum(params_p[k]*d[k] for k in d)
    return float(eP)

def get_HSPh(mol):
    d = get_dic_h(mol)
    if set(d) - set(params_h): return float(0)
    eH = sum(params_h[k]*d[k] for k in d)
    return float(eH)

def get_atom_code(atom):
    """ return an atom code consistent with the keys of the 'Vinc' dictionary """ 
    # be careful to take into account nD = number of deuterium atoms !
    nD = len([x for x in atom.GetNeighbors() if x.GetMass()>2 and x.GetSymbol()=='H'])
    if nD > 0:
        print('nD %u %s' %(nD, arg))
    code = atom.GetSymbol() + str(atom.GetTotalDegree()) + str(atom.GetTotalNumHs()+nD)
    code += 'a'*int(atom.GetIsAromatic())

    ##############################################################
    # Atribui o nome construído aqui à etiqueta de nome do átomo:
    atom.SetProp("atomNote", code)
    ##############################################################
    return code

def get_Vm_RD(mol):         
    atoms = Counter(get_atom_code(atom) for atom in mol.GetAtoms())
    
    ##########################################################
    #print("Átomos nomeados para Vm e Rm: ", atoms)
    #etiquetas = dict(atoms)
    #[print(chave) for chave in etiquetas]

    #########################################################
    
    missing_atoms = set(atoms) - set(vi)
    if missing_atoms: return None, None
    rings = get_ring_descriptors(mol)
    
    #########################################################
  #  print("Anéis considerados em Vm: ", rings)
    #########################################################

    Vm = sum([atoms[k]*vi[k] for k in atoms])   # atoms contribution
    Vm += sum([rings[k]*vi[k] for k in rings])  # rings contribution
    RD = sum([atoms[k]*ri[k] for k in atoms])   # molar refraction
    return Vm, RD

# UTILITY FOT OUTPUT
def tostr(energy, volume=None):
    if None in (energy, volume): return float(0)
#    if volume == 0: return " %5.1f " %energy
#    return " %5.1f " %sqrt(max(energy/volume,0))
    if volume == 0: return float(energy)
    return float(sqrt(max(energy/volume,0)))


def calcula(mol):
    mol = mol    
    # compute molar volume and refractivity
    Vm, RD = get_Vm_RD(mol)
    # compute D-component of HSP
    if Vm is None or RD is None:
        hspD = None
    else:
        hspD = sqrt(93.8 + (2016 + 75044./Vm) * (RD/Vm)**2)
    # compute P-component of HSP
    eP = get_HSPp(mol)
    eH = get_HSPh(mol)
    # convert energies to HSP
    hspD, hspP, hspH =  tostr(hspD,0), tostr(eP,Vm), tostr(eH,Vm)
    return Vm, RD, hspD, hspP, hspH

In [9]:
# a lista de moléculas no set de análise é a lista na variável Smiles. Neste caso, o conjunto de substâncias no estudo
smiles = [
          ['CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4',	'Nevirapine anhydrous',	'129618-40-2'],
          ['C1=CC=C(C=C1)C(=O)O',	'benzoic acid',	'65-85-0'],
          ['C1=CC=C(C(=C1)C(=O)O)O',	'salicylic acid',	'69-72-7'],
          ['C1=CC(=CC(=C1)O)C(=O)O',	'3-Hydroxybenzoic acid	99-06-9'],
          ['C1=CC(=CC=C1C(=O)O)O',	'4-hydroxybenzoic acid',	'99-96-7'],
          ['C1=CC(=C(C=C1O)C(=O)O)O',	'2,5-dihydroxybenzoic acid',	'490-79-9'],
          ['C1=CC(=C(C=C1O)O)C(=O)O',	'2,4-dihydroxybenzoic acid',	'89-86-1'],
          ['COCC1COC2=C(O1)C=C3C(=C2)N=CN=C3Cl',	'4-chloro-7-(methoxymethyl)-7,8-dihydro-[1,4]dioxino[2,3-g]quinazoline',	'SCHEMBL6795035'],
          ['CC1=CC=CC=C1',	'toluene',	'108-88-3'],
          ['C1#CO1',	'epoxy acetylene',	'21871629'],
          ['CCOC(=O)C',	'ethyl acetate',	'141-78-6'],
          ['CCCCO',	'butan-1-ol',	'71-36-3'],
          ['CCCCCCO',	'hexan-1-ol',	'8103'],
          ['CCCCCCCO',	'heptan-1-ol',	'111-70-6'],
          ['CCCCCCCCO',	'octan-1-ol',	'111-87-5'],
          ['CCCO',	'propan-1-ol',	'71-23-8'],
          ['CCCCCO',	'pentan-1-ol',	'71-41-0'],
          ['CC#N',	'acetonitrile',	'75-05-8'],
          ['C1CCC(CC1)C(=O)N2CC3C4=CC=CC=C4CCN3C(=O)C2',	'2-(cyclohexanecarbonyl)-3,6,7,11b-tetrahydro-1H-pyrazino[2,1-a]isoquinolin-4-one',	'55268-74-1'],
          ['C(=O)(N)N', 'urea', '57-13-6'],
          ['CN1C=NC2=C1C(=O)N(C(=O)N2C)C', '1,3,7-trimethylpurine-2,6-dione', '58-08-2'],
          ['CN1C=NC2=C1C(=O)NC(=O)N2C', '3,7-dimethylpurine-2,6-dione', '83-67-0']
          ]

In [10]:
# Calcula para toda a lista 'smiles' e inclui em um dataframe Pandas


# Cria uma lista de listas onde serão armazenados os valores calculados:
dataset = [['Mol_Image', 'nome_mol','smile', 'Vm', 'Rm', 'hspD', 'hspP','hspH']]

#Lê cada entrada na lista de substâncias e calcula para todas elas:
for entry in smiles:
    molsrc = entry[0]     # read molecule
    nome_mol = entry[1]
    mol = Chem.MolFromSmiles(molsrc)
    Vm, RD, hspD, hspP, hspH = calcula(mol)
    # Verifica se o retorno da calculadora é nulo, através da variável representativa Vm e apensa cada novo calculado à dataset
    if Vm is not None:
        #line = molname.ljust(12) + hspD + hspP + hspH + " %7.1f" %Vm
        #line = "hspD, hspP, hspH, Vm, distância_NVP, RED: " + hspD + hspP + hspH + " %7.1f" %Vm + str(Ra)+ str(RED)
        #dataset.append([Chem.MolFromSmiles(molsrc), nome_mol, molsrc, Vm, RD, hspD, hspP,hspH, distancia_NVP, RED])
        dataset.append([Chem.MolFromSmiles(molsrc), nome_mol, molsrc, Vm, RD, hspD, hspP,hspH])
    else:
        #line = molname.ljust(12) + hspD + hspP + hspH + "   None"
        line = "hspD, hspP, hspH, Vm" + hspD + hspP + hspH + "   None"
        dataset.append([Chem.MolFromSmiles(molsrc), nome_mol, molsrc, 0, RD, hspD, hspP,hspH])
    #print(dataset)

# Cria o DataFrame Pandas com os dados calculados das substâncias
df = pd.DataFrame(dataset[1:], columns=dataset[0])
# print dataframe.
df.head()


,Mol_Image,nome_mol,smile,Vm,Rm,hspD,hspP,hspH
0,"<img data-content=""rdkit/molecule"" src=""data:i...",Nevirapine anhydrous,CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4,199.78,74.57,20.664235,0.000000,10.096764
1,"<img data-content=""rdkit/molecule"" src=""data:i...",benzoic acid,C1=CC=C(C=C1)C(=O)O,101.53,33.28,19.743844,6.187427,10.491938
2,"<img data-content=""rdkit/molecule"" src=""data:i...",salicylic acid,C1=CC=C(C(=C1)C(=O)O)O,100.08,34.81,20.698098,8.947399,17.335789
3,"<img data-content=""rdkit/molecule"" src=""data:i...",3-Hydroxybenzoic acid\t99-06-9,C1=CC(=CC(=C1)O)C(=O)O,100.08,34.81,20.698098,8.947399,17.335789
4,"<img data-content=""rdkit/molecule"" src=""data:i...",4-hydroxybenzoic acid,C1=CC(=CC=C1C(=O)O)O,100.08,34.81,20.698098,8.947399,17.335789


In [11]:
# Define a molécula alvo do estudo, onde está o centro da esfera de Hansen e o raio da esfera
R_alvo = 7.0 # Chute inicial pode ser 10. Pode-se usar a planilha de fit do excel (Abott). Deve-se criar uma função fit pro raio.
mol_alvo = ['CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4',	'Nevirapine anhydrous',	'129618-40-2']

# A partir do dataframe, calcula a esfera de Hansen para uma dada substância alvo em relação às demais substâncias do dataframe.
def Hansen(mol_alvo, R_alvo):

   # Calcula os parâmetros para a molécula alvo
    VM_alvo, RD_alvo, hspD_alvo, hspP_alvo, hspH_alvo = calcula(Chem.MolFromSmiles(mol_alvo[0]))

    # Cria novas colunas em um novo data-frame à partir do dataframe original
    new_df=df.assign(Ra = ((4*(hspD_alvo - df['hspD'])**2+(hspP_alvo-df['hspP'])**2+(hspH_alvo-df['hspH'])**2)**0.5),  
                     RED = ((4*(hspD_alvo - df['hspD'])**2+(hspP_alvo-df['hspP'])**2+(hspH_alvo-df['hspH'])**2)**0.5)/R_alvo)
    return [new_df, VM_alvo, RD_alvo, hspD_alvo, hspP_alvo, hspH_alvo]
    
new_df = Hansen(mol_alvo, R_alvo)
VM_alvo = new_df[1]
RD_alvo = new_df[2]
hspD_alvo = new_df[3]
hspP_alvo = new_df[4]
hspH_alvo = new_df[5]

df1 = pd.DataFrame(new_df[0])
df1.head()
df = df1


In [12]:
fig = px.scatter_3d(df, x='hspD', y='hspP', z='hspH', size = 'Vm', color='RED', hover_data=['nome_mol'], color_continuous_scale=px.colors.sequential.Inferno)
fig.update_layout(scene_aspectmode='cube')   # fix the ratio in the top left subplot to be a cube

In [34]:
fig2 = px.scatter(df, x='hspD', y='hspP', size='Vm', color='RED', hover_data=['nome_mol'], labels='Rm', color_continuous_scale=px.colors.sequential.Jet)
fig3 = px.scatter(df, x='hspD', y='hspH', size='Vm', color='RED', hover_data=['nome_mol'], labels='Rm', color_continuous_scale=px.colors.sequential.Jet)
fig4 = px.scatter(df, x='hspP', y='hspH', size='Vm', color='RED', hover_data=['nome_mol'], labels='Rm', color_continuous_scale=px.colors.sequential.Jet)
fig5 = px.scatter_ternary(df, a='hspD', b='hspP', c='hspH', size='Vm', color='RED', hover_data=['smile'], labels='Rm', color_continuous_scale=px.colors.sequential.Jet)

In [35]:
# Add circles and configure the graph

fig2.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

fig2.add_shape(type="circle",
    xref="x", yref="y",
    x0 = hspD_alvo - R_alvo, y0 = hspP_alvo - R_alvo,
    x1 = hspD_alvo + R_alvo, y1 = hspP_alvo + R_alvo,
    #opacity=0.1,
    #fillcolor="orange",
    line_color="black",
)

fig2.update_layout(width=800, height=800)
fig2.show()

In [36]:
fig3.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

fig3.add_shape(type="circle",
    xref="x", yref="y",
    x0 = hspD_alvo - R_alvo, y0 = hspH_alvo - R_alvo,
    x1 = hspD_alvo + R_alvo, y1 = hspH_alvo + R_alvo,
    #opacity=0.1,
    #fillcolor="orange",
    line_color="black",
)

fig3.update_layout(width=800, height=800)

fig3.show()

In [37]:
fig4.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

fig4.add_shape(type="circle",
    xref="x", yref="y",
    x0 = hspP_alvo - R_alvo, y0 = hspH_alvo - R_alvo,
    x1 = hspP_alvo + R_alvo, y1 = hspH_alvo + R_alvo,
    #opacity=0.1,
    #fillcolor="orange",
    line_color="black",
)

fig4.update_layout(width=800, height=800)

fig4.show()